In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
plt.style.use(['dark_background'])

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from keras.layers import Input, Conv2D, Conv1D, AveragePooling1D, AveragePooling2D, Dense, Flatten, Dropout, BatchNormalization, MaxPool1D, Activation, concatenate
from keras.models import Model
import keras

from keras.regularizers import l2

import keras


debug_mode = 1

from clr import CyclicLR
from sklearn.externals import joblib
from sklearn.model_selection import KFold

import gc 

import tensorflow as tf
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)


Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
##############
NAME_OF_NOTEBOOK = 21
##############

In [3]:
def model():
    # This returns a tensor
    inputs = Input(shape=(55,300))
    input_head = Input(shape=(6,))

    x = Dense(256)(inputs)
    x = Activation('elu')(x)
    
    x = Dense(256)(x)
    x = Activation('elu')(x)
    
    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)

    x = Dropout(0.2)(x)
    x = Flatten()(x)

    # a layer instance is callable on a tensor, and returns a tensor
    x2 = Dense(16)(input_head) 
    #x2 = BatchNormalization()(x2)
    x2 = Activation('elu')(x2)

    x = concatenate([x,x2])
    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)
    x = Dropout(0.2)(x)

    x = Dense(64, activation='relu')(x)
    #x = Dense(128, activation='relu')(x)
    predictions = Dense(55, activation='linear')(x)

    # This creates a model that includes
    # the Input layer and three Dense layers
    model = Model(inputs=[inputs,input_head], outputs=predictions)

    model.compile(optimizer='nadam',
                  loss='mse',
                  )
    return model 


In [4]:
def load_data():
    X_train_head = np.load('all_train_heads.npy')
    X_train = np.load('all_train.npy')
    Y = np.load('all_targets.npy')
    return X_train_head, X_train, Y

# def load_test_data():
#     X_test_head = np.load('all_test_heads.npy')
#     scaler_filename = "scaler.save"
#     sk = joblib.load(scaler_filename)
#     X_test_head = sk.transform(X_test_head)

#     X_test = np.load('all_test.npy')
#     X_test = X_test.reshape(-1,55*300)
#     X_test = (X_test-1)*100
#     X_test = X_test.reshape(-1,55,300)
#     return X_test_head, X_test



In [5]:
X_train_head, X_train, Y = load_data()

In [6]:
X_train = X_train.reshape(-1,55*300)
X_train = (X_train-1)*1000
X_train = X_train.reshape(-1,55,300)

sk = StandardScaler()
scaler_filename = "scaler.save"
X_train_head = sk.fit_transform(X_train_head)
joblib.dump(sk, scaler_filename) 


['scaler.save']

In [7]:
# X_test_head, X_test = load_test_data()
from datetime import datetime


In [8]:
def CV_keras(X_train, X_train_head, Y, train=True):
    
    global NAME_OF_NOTEBOOK
    
    kf = KFold(n_splits=10, random_state=23, shuffle=True)
    mse_scores = []
    for j, (train_idx, val_idx) in enumerate(kf.split(X_train_head)):
        #print("TRAIN:", train_idx, "TEST:", val_idx)
        #x_train, x_train_head, x_test, x_test_head = X_train[train_idx], X_train_head[train_idx], X_train[val_idx], X_train_head[val_idx]
        #y_train, y_test = Y[train_idx]*1000, Y[val_idx]*1000

        # get a model
        mdl = model()

        debug_mode = 1
        
        whts_file = "./weights/{}_CV_weights_fold_{}.h5".format(NAME_OF_NOTEBOOK, j)

        if train:
            # calbacks
            logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
            tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
            early_stopping = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 23, verbose = debug_mode, mode= 'min')
            checkpoint = keras.callbacks.ModelCheckpoint(whts_file, monitor='val_loss', verbose = debug_mode, save_best_only=True, mode='min')

            clr = CyclicLR(base_lr=0.0009, max_lr=0.03,
                                    step_size=301., mode='triangular2')
            calls = [tensorboard_callback, early_stopping, checkpoint, clr, keras.callbacks.TerminateOnNaN()]
        
            mdl.fit([ X_train[train_idx], X_train_head[train_idx]], Y[train_idx]*1000, epochs=1000, batch_size=3048,
                  validation_data=([ X_train[val_idx], X_train_head[val_idx]], Y[val_idx]*1000), callbacks=calls)

        # load best weights 
        mdl.load_weights(whts_file)

        # prediction on validation dataset
        y_pred = mdl.predict([X_train[val_idx], X_train_head[val_idx]])
        score = mean_squared_error(Y[val_idx]*1000, y_pred)
        mse_scores.append(score)
        print("FOLD-{}, MSE: {} \n".format(j+1, score))

        # save some place 
        del y_pred, mdl
        gc.collect()
    print("MSE ", mse_scores)
    print('\n AVERAGE MSE: ', np.mean(mse_scores))


In [10]:
CV_keras(X_train, X_train_head, Y, train=False)

FOLD-1, MSE: 10.209456952296684 

FOLD-2, MSE: 10.718903233534231 

FOLD-3, MSE: 9.838143423048567 

FOLD-4, MSE: 9.216341205803587 

FOLD-5, MSE: 9.788903213255434 

FOLD-6, MSE: 8.521990591126913 

FOLD-7, MSE: 10.567978675195748 

FOLD-8, MSE: 8.688548324320623 

FOLD-9, MSE: 8.738196119911331 

FOLD-10, MSE: 10.056875032708389 

MSE  [10.209456952296684, 10.718903233534231, 9.838143423048567, 9.216341205803587, 9.788903213255434, 8.521990591126913, 10.567978675195748, 8.688548324320623, 8.738196119911331, 10.056875032708389]

 AVERAGE MSE:  9.634533677120151
